<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
#!python -m wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

In [ ]:
#!pip install gensim

In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


Найдем наиболее близкие слова к слову `dog`:

In [ ]:
print(f"Num of words: {len(wv_embeddings.index_to_key )}")

Num of words: 1787145


#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?
* Ответ: Слова 'cat' нет, но есть слово 'cats' на 4 месте


In [ ]:
# method most_simmilar

wv_embeddings.most_similar('dog')

[('animal', 0.8564180135726929),
 ('dogs', 0.7880866527557373),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665),
 ('feline', 0.7392398715019226),
 ('bird', 0.7315488457679749),
 ('animal1', 0.7219215631484985),
 ('doggy', 0.7213349342346191),
 ('labrador', 0.7209131717681885)]

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
import numpy as np
import re
# you can use your tokenizer
#from nltk.tokenize import WordPunctTokenizer

class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

In [ ]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    tokenized_question = tokenizer.tokenize(text=question.lower())

    vec = np.zeros(dim)
    count = 0

    for word in tokenized_question:
        if word in embeddings:
            vec += embeddings[word]
            count+=1

    if count != 0:
        vec = vec / count

    return vec


question_to_vec('I love neural networks', wv_embeddings, tokenizer, 200)

array([-1.01422749, -1.68912624, -1.28541217, -1.37103013,  0.15916634,
        0.0670696 ,  1.11884137, -0.07051762,  0.53863693, -0.37139114,
        0.28913016,  1.03125   , -2.86546615, -1.31928726,  2.20982948,
        1.16786722,  1.08419518,  1.84672268, -0.98733809,  0.87209455,
       -0.17219625,  0.73839017, -0.78278264, -0.88278695, -0.61526108,
       -0.15872356, -0.61707981,  0.36668165,  2.01375335,  0.26274722,
       -1.4950124 , -1.38671508, -1.20499369, -1.01661696,  2.76211715,
        1.72557124, -0.19038178, -0.58994474, -0.1141155 , -1.88903529,
       -1.78835048, -0.20420383, -0.70620341,  2.4890339 , -0.15452717,
        1.12550129,  0.26108352,  1.31532462, -0.57667526, -0.33271411,
        1.67649019,  0.91229995, -0.6988014 ,  2.37525521, -1.76342229,
        3.13780514,  1.22432033, -1.05544141,  0.78975705,  0.04404024,
        1.16230452, -0.44780231, -0.9149816 ,  0.01501505, -0.85523087,
       -1.01834532, -0.71621163,  1.0727205 ,  0.88818762, -0.29

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [ ]:
round(question_to_vec('I love neural networks', wv_embeddings, MyTokenizer(), 200)[2], 2)

-1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
\text{Первой простой метрикой будет количество корректных попаданий для какого-то } K:
$$
\text{Hits@}K = \frac{1}{N} \sum_{i=1}^N [\text{rank}_{q_i'} \leq K],
$$

где

$$
[x < 0 ] \equiv
\begin{cases}
   1, & \text{если } x < 0 \\
   0, & \text{если } x \geq 0
\end{cases}
$$
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1` = 1


<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [ ]:
round(1/np.log2(1+9),1)

0.3

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [ ]:
def hits_count(dup_ranks, k):
    hits_value = sum([1 if rank <= k else 0 for rank in dup_ranks])/len(dup_ranks)
    return hits_value

In [ ]:
def dcg_score(dup_ranks, k):
    dcg_value = sum([1/np.log2(1+rank) if rank <= k else 0 for rank in dup_ranks])/len(dup_ranks)
    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity([question_to_vec("How does the catch keyword determine the type of exception that was thrown", wv_embeddings, MyTokenizer(), 200)], \
                  [question_to_vec("how does the catch keyword determine the type of exception that was thrown", wv_embeddings, MyTokenizer(), 200)])[0][0]

1.0

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1

max_cosine_similarity = 0
copy_answers_vec = question_to_vec(copy_answers[0], wv_embeddings, MyTokenizer(), 200)
copy_rank = 0

for index, candidate in enumerate(candidates_ranking[0]):
    candidate_vec = question_to_vec(candidate, wv_embeddings, MyTokenizer(), 200)
    cosine_similarity_candidate = cosine_similarity([candidate_vec], [copy_answers_vec])[0][0]
    if cosine_similarity_candidate > max_cosine_similarity:
        max_cosine_similarity = cosine_similarity_candidate
        copy_rank = index + 1
        copy_rank_index = index


dup_ranks = [copy_rank]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
!unzip stackoverflow_similar_questions.zip

"unzip" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


Считайте данные.

In [ ]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        line  = line.split('\t')
        line[-1] = line[-1][:-1]
        data.append(line)
    return data

Нам понадобиться только файл validation.

In [ ]:
validation_data = read_corpus('validation.tsv')

Кол-во строк

In [ ]:
len(validation_data)

3760

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [ ]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    tuple_candidates = tuple(enumerate(candidates))
    question_vec = question_to_vec(question, embeddings, tokenizer, dim)
    res = []
    for index, candidate in tuple_candidates:
        candidate_vec = question_to_vec(candidate, embeddings, tokenizer, dim)
        cosine_similarity_candidate = cosine_similarity([candidate_vec], [question_vec])[0][0]
        res.append([cosine_similarity_candidate, (index, candidate)])
    res.sort(reverse=True)
    return [item[1] for item in res]

In [ ]:
rank_candidates('labrador', ['horse','engine','dog'], wv_embeddings, MyTokenizer())

[(2, 'dog'), (0, 'horse'), (1, 'engine')]

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'),
            (*, 'select2 not displaying search results'),
            (*, 'WPF- How to update the changes in list item of a list')]]

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

# Ответ: 021

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
wv_ranking = []
max_validation_examples = 1000

for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.402 | Hits@   1: 0.402
DCG@   5: 0.498 | Hits@   5: 0.582
DCG@  10: 0.519 | Hits@  10: 0.650
DCG@ 100: 0.565 | Hits@ 100: 0.874
DCG@ 500: 0.578 | Hits@ 500: 0.973
DCG@1000: 0.581 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [ ]:
train_data = read_corpus('train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [ ]:
sequences = [' '.join(lines) for lines in train_data]

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [ ]:
for seq_index, sequence in enumerate(sequences):
    sequence = tokenizer.tokenize(sequence.lower())
    new_sequence = [word for word in sequence if word not in stop_words]
    sequences[seq_index] = new_sequence

In [ ]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(sequences, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=10,             # consider words that occured at least 5 times
                 window=5).wv

Размер window равен 5 т.к. это оптимальный размер для захвата контекста слова без учета слишком далеких, что позволяет модели лучше улавливать семантические связи между словами.

In [ ]:
embeddings_trained.index_to_key

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.404 | Hits@   1: 0.404
DCG@   5: 0.505 | Hits@   5: 0.592
DCG@  10: 0.527 | Hits@  10: 0.662
DCG@ 100: 0.572 | Hits@ 100: 0.879
DCG@ 500: 0.585 | Hits@ 500: 0.978
DCG@1000: 0.587 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:

* Токенизаторы, такие как spaCy или nltk, дают качество лучше потому что они применяют сложные алгоритмы машинного обучения или правила, учитывающие контекст и языковые особенности, что позволяет точнее разделять текст на токены, особенно в случаях, где простые методы могут допускать ошибки.
* Да, использование нормализации слов, такой как приведение к нижнему регистру, помогает унифицировать текст и снизить размерность данных.
* Выбор эмбеддингов зависит от конкретной задачи: BERT и другие контекстуальные модели часто эффективны для задач семантического анализа, в то время как Word2Vec и GloVe подходят для более простых задач, таких как анализ тональности, благодаря учету семантических отношений между словами.
* Плохое качество решения задачи, возможно, объясняется использованием простого токенизатора и базовых эмбеддингов, а также отсутствием процессов стемматизации или лемматизации, что могло бы улучшить уровень обработки текста.
* Для улучшения качества решения задачи, я думаю стоит использовать более продвинутые методы токенизации, такие как spaCy или nltk, чтобы более точно учесть особенности языка. Также рассмотрите применение контекстуальных эмбеддингов, которые способны улавливать сложные семантические зависимости в тексте. Дополнительно, применение стемматизации или лемматизации может улучшить обработку различных форм слов и снизить размерность данных.
